In [4]:
from fitting import FitModel
from databases import initiate_db, print_summary
from plotters import forces_plots
from convergence_checker import get_modes, converge_check
from scipy import optimize

In [6]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  +0.960,
                           'shell': -2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }

params['cs_springs'] = {'O' : [100.0, 0.0]}

distribution = {}
distribution['Li-O'] = {'bpp' : [691.229, 0.269, 0.0],
                        'sd' : [80, 0.01, 0.01]}

distribution['Ni-O'] = {'bpp' : [591.665, 0.382, 0.000],
                        'sd'  : [80, 0.01, 0.01]}

distribution['O-O'] = {'bpp' : [22739.211, 0.146, 20.7],
                       'sd'  : [200, 0.01, 5]}

excude_from_fit = [] # string of atom1_atom2_param. Example of format = 'O_O_rho'

In [ ]:
i=1
prev_modes = None
converge = False
# summary_filename = 'summary.csv'
# initiate_db(summary_filename)

while converge is False:
    
    #updates the potentials in the distribution dictionary with mean values
    distribution = update_potentials(trace, modes, distribution)
    
    #Runs with mode potential and returns forces
    kwargs = mode_potentials(trace)
    mode_forces = fit_data.get_forces(**kwargs)
    
    #Fills the databases
#     print_summary(i, trace, summary_filename)
    
    #Plots the distributions
#     forces_plots(forces_filename, i)
    i+=1

In [7]:
fit_data = FitModel.collect_info(params, distribution, supercell=[1,1,1])

Found elements: ['Li', 'Ni', 'O']


In [22]:
keys = []
values = []
for potential in fit_data.potentials:
    keys.append(potential.a.label_string)
    keys.append(potential.rho.label_string)
    keys.append(potential.c.label_string)
    values.append(potential.a.value)
    values.append(potential.rho.value)
    values.append(potential.c.value)

In [24]:
optimize.minimize(error, values, args=keys)

Signature:
optimize.minimize(
    fun,
    x0,
    args=(),
    method=None,
    jac=None,
    hess=None,
    hessp=None,
    bounds=None,
    constraints=(),
    tol=None,
    callback=None,
    options=None,
)
Docstring:
Minimization of scalar function of one or more variables.

Parameters
----------
fun : callable
    The objective function to be minimized.

        ``fun(x, *args) -> float``

    where x is an 1-D array with shape (n,) and `args`
    is a tuple of the fixed parameters needed to completely
    specify the function.
x0 : ndarray, shape (n,)
    Initial guess. Array of real elements of size (n,),
    where 'n' is the number of independent variables.
args : tuple, optional
    Extra arguments passed to the objective function and its
    derivatives (`fun`, `jac` and `hess` functions).
method : str or callable, optional
    Type of solver.  Should be one of

        - 'Nelder-Mead' :ref:`(see here) <optimize.minimize-neldermead>`
        - 'Powell'      :ref:`(see here)